In [32]:
import pandas as pd
import pyarrow.parquet as pq

In [33]:
DATA_DIR = '../data/'
anonym_http = pq.read_table(DATA_DIR + 'anonymousHTTPRequests-20180217.parquet').to_pandas()
bots = pq.read_table(DATA_DIR + 'botsUidBidTable.parquet').to_pandas()

In [34]:
anonym_http = anonym_http.sample(frac=0.003)

In [35]:
anonym_http.head()

,browserId,from,ip,method,operation,referrer,requestType,timestamp,to,url,userAgent,userId
469675,5.392760e+18,None,DxxUC9pRdt6de12zAfvcOg==,GET,WidgetShare,http://www.o-my-baby.ru/zdorovie/lekarstva/noo...,NAV,1518760487437,WidgetShare,https://connect.ok.ru/dk,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,null
406530,6.073509e+18,None,Mfx14qVZ4K89THmzQVMhpA==,GET,spring-mvc,https://ok.ru/,REQ,1518801192461,None,https://ok.ru/web-api/pts/reactions.client,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,null
198658,8.906670e+18,None,LweVoFleSDB3UiuvHCjjWw==,None,None,None,UPGRADE,1518803463181,None,http://ok.ru/dk?st.cmd=shareData&ref=http%3A%2...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,null
441305,NaN,None,hmzdccT9NzomUmTOC5ZTKw==,POST,spring-mvc,https://ok.ru/sw.js,REQ,1518811900941,None,https://ok.ru/web-api/webpush/log,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,null
220540,NaN,None,WhPSSj7d+E40X8x77R3yVA==,GET,anonymVideoEmbed,http://www.replaymatches.com/2018/02/borussia-...,NAV,1518766803469,anonymVideoEmbed,https://www.ok.ru/videoembed/670003235436,Mozilla/5.0 (iPhone; CPU iPhone OS 11_2_5 like...,null


### Сделаем sparse matrix 

In [36]:
print('имеем {} уникальных операций'.format(anonym_http['operation'].unique().shape[0]))

имеем 76 уникальных операций


In [37]:
# закодируем операции интами
dict_op = {}

counter = 0
for op in anonym_http['operation'].unique():
    dict_op[op] = counter
    counter += 1

In [38]:
anonym_http['requestType'].unique()

array(['NAV', 'REQ', 'UPGRADE', 'CSP'], dtype=object)

In [39]:
anonym_http['requestType'].value_counts()

REQ        1771
NAV        1199
UPGRADE     476
CSP          32
Name: requestType, dtype: int64

In [40]:
# запилю сразу requestType
# dict_req = {}

# c = 0
# for rt in anonym_http['requestType']:
#     dict_req[rt] = c
#     c += 1
    
# anonym_http['requestType'] = anonym_http['requestType'].apply(lambda x: dict_req[x])    

In [41]:
anonym_http['operation'] = anonym_http['operation'].apply(lambda x: dict_op[x]);

In [42]:
from sklearn.preprocessing import OneHotEncoder

In [43]:
enc = OneHotEncoder(sparse=True)

In [44]:
encoded_categorical_columns = pd.DataFrame(enc.fit_transform(anonym_http[['operation']]).toarray())
encoded_categorical_columns.head()

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
anonym_http.index = range(len(anonym_http.index))

In [46]:
#encoded_categorical_columns.columns = encoded_categorical_columns.columns.astype(str)

In [50]:
anonym_http.join(encoded_categorical_columns)

,browserId,from,ip,method,operation,referrer,requestType,timestamp,to,url,...,66,67,68,69,70,71,72,73,74,75
0,5.392760e+18,None,DxxUC9pRdt6de12zAfvcOg==,GET,0,http://www.o-my-baby.ru/zdorovie/lekarstva/noo...,NAV,1518760487437,WidgetShare,https://connect.ok.ru/dk,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6.073509e+18,None,Mfx14qVZ4K89THmzQVMhpA==,GET,1,https://ok.ru/,REQ,1518801192461,None,https://ok.ru/web-api/pts/reactions.client,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8.906670e+18,None,LweVoFleSDB3UiuvHCjjWw==,None,2,None,UPGRADE,1518803463181,None,http://ok.ru/dk?st.cmd=shareData&ref=http%3A%2...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NaN,None,hmzdccT9NzomUmTOC5ZTKw==,POST,1,https://ok.ru/sw.js,REQ,1518811900941,None,https://ok.ru/web-api/webpush/log,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NaN,None,WhPSSj7d+E40X8x77R3yVA==,GET,3,http://www.replaymatches.com/2018/02/borussia-...,NAV,1518766803469,anonymVideoEmbed,https://www.ok.ru/videoembed/670003235436,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2.610366e+18,None,FYURESp1bU8Txz9yjrwB6Q==,POST,4,https://ok.ru/videoembed/280611850781,REQ,1518743997453,None,https://ok.ru/dk,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.956856e+18,None,bsy8dbZsZf2zbVHHkJAJnA==,None,2,None,UPGRADE,1518799972877,None,http://www.odnoklassniki.ru/dk?st.cmd=browserT...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,-9.155728e+18,None,ryqos8UMWORntaxTNIv3PQ==,GET,1,https://ok.ru/,REQ,1518772885517,None,https://ok.ru/web-api/pts/video.player,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,-1.758600e+18,None,0K3o2GViFsVbpO3r4x3Obg==,POST,4,https://www.ok.ru/videoembed/328449460754,REQ,1518738848781,None,https://www.ok.ru/dk,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2.885770e+18,None,AAMI33xK4e+50DuyCawKSQ==,POST,4,https://ok.ru/,REQ,1518755550221,None,https://ok.ru/dk,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# for s in encoded_categorical_columns.columns:
#     anonym_http[s] = encoded_categorical_columns[s]

In [66]:
# anonym_http['new'] = encoded_categorical_columns['0']

In [92]:
# encoded_categorical_columns['0'].head(10)